<a href="https://colab.research.google.com/github/dorinhazan/FinalProject-DataScience/blob/main/observable_extraction_assistant_and_observable_rank_validator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! unzip /content/validation.zip

Archive:  /content/validation.zip
   creating: validation/
  inflating: __MACOSX/._validation   
  inflating: validation/results-reports-new-prompt.json  
  inflating: __MACOSX/validation/._results-reports-new-prompt.json  
  inflating: validation/%5BAnalysis%5DAndariel_Group.md  
  inflating: __MACOSX/validation/._%5BAnalysis%5DAndariel_Group.md  


In [ ]:
%env OPENAI_API_KEY=""

env: OPENAI_API_KEY=""


In [ ]:
from openai import OpenAI
from google.colab import userdata
import os
from openai import OpenAI
import regex as re

api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key = api_key)

## Removing images from the Markdown files

In [ ]:
def clean_markdown(file_path):
    """
    Clean a single Markdown file by removing hyperlinks, annotations, headers, and footers.
    Args:
        file_path (str): Path to the Markdown file.
    """
    with open(file_path, "r", encoding="utf-8") as file:
        content = file.read()


    # Patterns for removal
    patterns = {
        "inline_hyperlinks": r'\[.*?\]\(http[s]?://\S+?\)',  # Matches [text](http://example.com)
        "standalone_urls": r'http[s]?://\S+',  # Matches http://example.com
        # "annotations": r'\[\]\(\S+\)',  # Matches empty image annotations like []()
        "headers_footers": r'(Recommended Practice|Homeland Security|Page \d+)',  # Matches headers/footers
        "images": r'!\[[^\]]*\]\((?:[^()\\]|\\.)*?\)', # Matches image anotation like ![alt text](url “optional title”)
    }

    # Debugging: Print content before cleaning

    for name, pattern in patterns.items():
        matches = re.findall(pattern, content)
        print(matches)
        content = re.sub(pattern, '', content)


    # Save the cleaned content
    with open(file_path, "w", encoding="utf-8") as file:
        file.write(content)


In [ ]:
clean_markdown("/content/validation/%5BAnalysis%5DAndariel_Group.md")

[]
[]
[]
[]


## Corected Code - gpt_for_reports.py


In [ ]:
obs_rank_validator_instructions = """

# Observable-rank Validator
 You are a senior CTI analyst. Your job is to **audit one observable at a time** and judge whether its `observable_rank` value (“Actionable”, “Described”, or “Mentioned”) is correct, based on the observable’s name and the provided context.
>
> You will receive:
> 1. The observable record (as JSON).
> 2. The surrounding sentence or paragraph from the original CTI snippet for minimal context (when available).

### 1.  Decision Rules

| Rank              | Decide “Actionable” **when …**                                                                                                                   | Typical examples (not exhaustive)                                                                         |
|-------------------|--------------------------------------------------------------------------------------------------------------------------------------------------|-----------------------------------------------------------------------------------------------------------|
| **Actionable**    | • The exact string can be matched **as‑is** (or after a trivial transform like Base64 decode).<br>• A detection rule (IDS/YARA/SIEM, PLC logic, etc.) could key on it with low false‑positives.<br>• Uniqueness is obvious (or highly likely) from its syntax or entropy. | – Full code snippets / shell commands / SQL queries<br>– Complete URLs, URIs, or file paths<br>– IPs, domains, GUIDs, hashes (MD5/SHA‑256/SHA‑1, Cert thumbprints), JWTs, beacon keys<br>– Hard‑coded strings, mutexes, registry keys, ICS/SCADA tags, Modbus function codes<br>– Crypto keys, X.509 DN strings, email addresses, phone numbers used for C2 |
| **Described**     | • The details are not sufficient to act upon **or** the observable is not searchable, for example, due to ambiguity or missing information.<br>• Additional enrichment (lookup table, parameter substitution, memory carving, etc.) is required **and** is non‑trivial. | – “An HTTP POST to `/api/checkin?id=<random>`” (placeholder)<br>– “A PowerShell script that loads shellcode” (no code shown)<br>– “A DLL named similar to system files” |
| **Mentioned**     | • It is only referenced conceptually or generically and **cannot** be turned into a concrete detection even after enrichment.                    | – “Uses AES‑128” (algorithm only)<br>– “Makes RPC calls” (no function names)<br>– “Phishing email” |

### 2.  Key Heuristics

1. **Code fence = Actionable**
   Any observable whose `observable_value` is wrapped in triple back‑ticks is presumed actionable *unless* it is clearly pseudocode.

2. **Complete identifier = Actionable**
   A string that *looks complete* for its type (e.g., 36‑char GUID, 64‑hex SHA‑256, “10.0.0.5”, “plc_tag=3001:1”) is actionable.

3. **Indicator Strings**
   Strings such as `"S^Anonymous?"`, mutex names, task names, beacon keys, campaign IDs, etc., are actionable if they appear exactly.

4. **Encoded Artifacts**
   If a value is Base64/hex/URL‑encoded **and its decoded form is also supplied**, both entries are independently actionable.

5. **When unsure**
   Err toward **higher precision**:
   ```
   Actionable  >  Described  >  Mentioned
   ```
   If the observable *could plausibly* drive automated detection, choose **Actionable**.

### 3.  Output Format

Respond only with one of the following, for each observable:

```text
CORRECT
```

```text
INCORRECT — should be "<CorrectRank>" because <one‑sentence justification>, note: "<copy 'note' field from observable record>"
```x

***(No other words.)***

---

### 4.  Quick Examples for Output Format

| Observable (value + rank) | Validator output | Rationale |
|---------------------------|------------------|-----------|
| ```send_403070(0x1Au,0…)``` → **Described** | `INCORRECT — should be "Actionable" because it is a full, unique code snippet that can be matched directly.` | Rule 1 |
| `aHR0cHM6Ly9jdC5lemlsLmNvbS9wYXlsb2Fk` (encoded) → **Actionable** | `CORRECT` | Encoded, unique, searchable |
| `“a DLL named netutils.dll”` → **Mentioned** | `INCORRECT — should be "Described" because it gives a partial but still ambiguous filename pattern.` | lacks uniqueness |
| `AES‑128` → **Mentioned** | `CORRECT` | Only an algorithm, non‑searchable |

---

Use these instructions as the **sole evaluation rubric**.
"""

In [ ]:
obs_rank_validator_ID = "asst_ksYtNJ5GTGYvwNQS6liBckyN"
our_updated_assistant = client.beta.assistants.update(
  obs_rank_validator_ID,
  instructions=obs_rank_validator_instructions,
)
# Notice! this code will update the assistant in OpenAI, so next time you will run the code at the beggining of this notebook, in will call the updated assistant

In [ ]:
from openai import OpenAI

# obs_rank_validator_ID = "asst_ksYtNJ5GTGYvwNQS6liBckyN"

def observable_rank_validator(observables):

  """
  Validating your Observable Extraction using an observable-rank validator.

  This function uses an OpenAI assistant I built for you, named Observable-Rank Validator, its purpose
  is to help you to validate your result from the observable extractor assistant.
  This function receives a list of observables (a JSON list of observables, as resulted from the Observable Extractor Assistant),
  and Respond with one of the two:

    ```text
    CORRECT
    ```

    ```text
    INCORRECT — should be "<CorrectRank>" because <one‑sentence justification>
    ```
  for each observable in the list, according to their order.

  """
  payload = json.dumps(observables, ensure_ascii=False, indent=2)

  # 2️⃣ Optionally wrap in a ```json code-fence if that’s how the assistant was trained
  user_msg = (
      "Here is a JSON list of observables. "
      "Validate the `observable_rank` of each one.\n"
      "```json\n" + payload + "\n```"
  )

  # 3️⃣ Create the thread / run
  thread = client.beta.threads.create(
      messages=[{"role": "user", "content": user_msg}],
  )

  run = client.beta.threads.runs.create_and_poll(
      thread_id    = thread.id,
      assistant_id = obs_rank_validator_ID
  )

  if run.status != "completed":
      raise RuntimeError(f"Validator run ended with status {run.status}")

  reply = client.beta.threads.messages.list(thread_id=thread.id).data[0]
  return reply.content[0].text.value

In [ ]:
import os
import re
import json
import time
from openai import OpenAI

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# Define base directories
base_path = "/content/validation"
obs_extractor_ID = "asst_LFsVfvrlMTkrFFlr3qeNssLV"


def find_md_files(base_path):
    """Recursively find all .md files in the directory."""
    md_files = []
    for root, _, files in os.walk(base_path):
        for file in files:
            if file.endswith(".md"):
                md_files.append(os.path.join(root, file))
    return md_files

def process_md_file(md_file_path):
    """Read and convert .md file to plain text."""
    with open(md_file_path, 'r') as file:
        return file.read()

def split_into_sections(md_content):
    """
    Split the markdown content into logical sections using markdown headers.

    This function uses a regular expression to find headers (lines starting with '#' characters)
    and splits the content accordingly. If the file doesn't contain any headers, it returns the entire
    content as one section.
    """
    pattern = re.compile(r'^(#{1,6}\s.*)', re.MULTILINE)
    matches = list(pattern.finditer(md_content))
    sections = []
    if not matches:
        # No headers found, return the full content as one section.
        sections.append(md_content)
        # print(sections)
        return sections

    # Add any content before the first header as its own section if present.
    if matches[0].start() > 0:
        sections.append(md_content[:matches[0].start()])

    # Split content based on header positions.
    for i, match in enumerate(matches):
        start = match.start()
        end = matches[i + 1].start() if i + 1 < len(matches) else len(md_content)
        sections.append(md_content[start:end])
    print(sections)
    return sections

def send_to_openai(section):
  message_thread = client.beta.threads.create(
    messages=[
      {"role":"user", "content": section}
    ],
  )
  run = client.beta.threads.runs.create_and_poll(
    thread_id = message_thread.id, # the ID of the Thread generated above
    assistant_id = obs_extractor_ID # the ID of the Observable Extractor Assistant
  )
  if run.status == 'completed':
    messages = client.beta.threads.messages.list(
    thread_id= message_thread.id,
    )
    for message in messages.data:
      if message.role == "assistant":
        messg_id = message.id
        result = message.content[0].text.value
        print(result)
    return result

def remove_duplicates(observables_list):
    """Remove duplicate dictionaries from a list of observables."""
    unique = []
    seen = set()
    for item in observables_list:
        # Convert dictionary to a JSON string for a consistent, hashable representation.
        item_str = json.dumps(item, sort_keys=True)
        if item_str not in seen:
            seen.add(item_str)
            unique.append(item)
    return unique


# Locate all markdown files in the base directory.
md_files = find_md_files(base_path)
print(f"Found {len(md_files)} markdown files.")

results = {}
ranking_list = []
total_start_time = time.time()

# Process each markdown file
for md_file in md_files:
    print(f"\nProcessing file: {md_file}")
    try:
        md_content = process_md_file(md_file)
        sections = split_into_sections(md_content)
        print(f"File split into {len(sections)} sections.")

        # Initialize an aggregated result for the current file
        # aggregated_result = {
        #     "Actionable": [],
        #     "Described": [],
        #     "Mentioned": []

        # }

        # Process each section separately
        for idx, section in enumerate(sections):
            # Create a prompt for the current section including an identifier for the section.
            # section_prompt = f"{prompt_template}\n\nSection {idx + 1}:\n{section}"
            section_prompt  = f"Here is the text snippet: \n'''\n{section}\n'''"
            print(f"Processing section {idx + 1} of {len(sections)}")
            print(f"\n--------------------------------Section {idx+1}-----------------------------------\n {section}")
            print(f"\n--------------------------------results-----------------------------------\n")
            try:
                raw_response = send_to_openai(section_prompt)

                if not raw_response.strip():
                    raise ValueError(f"Empty response for section {idx + 1} in file {md_file}")

                # Clean the raw response by removing potential markdown formatting.
                cleaned_response = raw_response.strip("```").strip("json").strip()
                parsed_response = json.loads(cleaned_response)
                #print(parsed_response)
                # if "observables" in parsed_response:
                #     for obs in parsed_response["observables"]:
                #         category = obs.get("observable_rank")
                #         if category in aggregated_result:
                #             aggregated_result[category].append(obs)
                print(f'parsed_response type {type(parsed_response)}')
                print(f'parsed_response {parsed_response}')
                print(f'parsed_response[observables] {parsed_response["observables"]}')

                results[os.path.basename(md_file)] = parsed_response
                # print(f'results {results}')
                if parsed_response["observables"]:          # not empty
                  ranking = observable_rank_validator(parsed_response["observables"])
                  ranking_list.append(ranking)
                  print("Ranking:\n", ranking)
                else:
                    print("No observables extracted in this section.")

            except Exception as section_error:
                print(f"Error processing section {idx + 1} of {md_file}: {section_error}")
                continue

        # Remove duplicates from both observables lists.
        # aggregated_result["Actionable"] = remove_duplicates(aggregated_result["Actionable"])
        # aggregated_result["Described"] = remove_duplicates(aggregated_result["Described"])
        # aggregated_result["Mentioned"] = remove_duplicates(aggregated_result["Mentioned"])

        # results[os.path.basename(md_file)] = aggregated_result

    except Exception as e:
        print(f"Error processing file {md_file}: {e}")
        results[os.path.basename(md_file)] = {"error": str(e)}

total_end_time = time.time()
total_elapsed = total_end_time - total_start_time
print(f"\nTotal processing time: {total_elapsed:.2f} seconds.")

# Save the aggregated results to a JSON file.
output_file_path = "/content/results-reports-new-prompt.json"
with open(output_file_path, "w") as output_file:
    json.dump(results, output_file, indent=4)
print(f"Results saved to {output_file_path}.")

# 4. Save the result to a text file
output_path = "/content/ranking_results.txt"
with open(output_path, "w", encoding="utf-8") as out_f:
    out_f.write(ranking_list)



Found 1 markdown files.

Processing file: /content/validation/%5BAnalysis%5DAndariel_Group.md
['Analysis Report (الران\n\n', '# Targeted attacks by Andariel Threat Group, a subgroup of the Lazarus\n\n\n\n220, Pangyoyeok-ro, Bundang-gu, Seongnam-si, Gyeonggi-do, South Korea Tel: +82-31-722-8000 | Fax: +82-31-722-8901 | www.ahnlab.com | © AhnLab, Inc. All rights reserved.\n\n', "## Table of Contents\n\n| Overview |\n| --- |\n| Attack Vectors (Infection Routes). |\n| 1. Spear Phishing |\n| 2. Watering Hole (Active-X Vulnerability) |\n| 3. Central Manaqement Solution . |\n| 4. Supply Chain Attack . |\n| Attack Cases . |\n| Malware and Attack Tools . |\n| 1. Malware – Backdoor …………………………………………………………………………………………………………………………………………………………………………………………………………………………………………………………………………………………………… 11 |\n| 1.1) Aryan . |\n| 1.2) Gh0st RAT . |\n| 1.3) Rifdoor………………………………………………………………………………………………………………………………………………………………………………………………………………………………………………………………………………………………………………… 12 |\n| 1.4) Phandoor |\n| 1.5) 

KeyboardInterrupt: 

# Observable-Rank Validator instructions:

In [ ]:
# import json

# # 1. Load file
# file_path = "/content/results-reports-new-prompt.json"
# with open(file_path, 'r', encoding='utf-8') as f:
#     observables_data = json.load(f)

# # 2. Convert back to a JSON string for the validator
# observables_list = json.dumps(observables_data)

# # 3. Run the validator
# ranking = obsrevable_rank_validator(observables_list)
# print(ranking)

# # 4. Save the result to a text file
# output_path = "/content/ranking_results.txt"
# with open(output_path, "w", encoding="utf-8") as out_f:
#     out_f.write(ranking)


NCopy of gh0st3..: INCORRECT — should be "Described" because it is only a fragment and ambiguous rather than a complete, unique identifier, note: "NCopy of gh0st3."  
%s\cmd.exe /c echo | %s > %s: INCORRECT — should be "Described" because it is a command template with placeholders requiring enrichment rather than a fixed indicator, note: "%s\cmd.exe /c echo | %s > %s"  
Putty Link: INCORRECT — should be "Described" because the tool name is generic and lacks the uniqueness needed for automated detection, note: "Putty Link"  
c:\work>zcon <ip> <port>: INCORRECT — should be "Described" because it is a command usage template with placeholders rather than a fixed, matchable string, note: "c:\work>zcon <ip> <port>"  

All the remaining observables have the correct observable_rank.


# **Change Prompt Here (if needed)**

so according to the result of the code above, the code snippet (the 8th json in the list in the example), should be actionable. therefore, 1 out of the 17 observables in this list is incorrect and should be corrected.

Your job is to try to maximize the ability of the Observable Extractor Assistant by improving its prompt. In the given example the accuracy of the model is high, but in other examples there are alot of mistakes made by the model (rakning code as described instead of actionable and code template as actionable instead of described and such).

**you can change the Observable Extractor Assistant instructions using the code below:**

In [ ]:
our_new_observable_extractor_assistant_instructions = """
You are a helpful Cybersecurity assistant for identifying observables in Cyber Threat Intelligence text snippets.

---
### Task

1. You will receive a text snippet of a CTI report from a user.
2. Read the given snippet (plain text) carefully.
3. Extract *every observable* (artifact) mentioned — do *not* omit any.
4. For each observable, output a JSON object with the exact fields listed in the *Response format* section.

---

### Decision Rules

| Rank           | Decide “Actionable” **when …**                                                                                                                                      | Typical examples                                                                                                              |
| -------------- | ------------------------------------------------------------------------------------------------------------------------------------------------------------------- | ----------------------------------------------------------------------------------------------------------------------------- |
| **Actionable** | • The exact string can be matched **as‑is** (or after a trivial transform like Base64 decode).<br>• A detection rule (IDS/YARA/SIEM, PLC logic, etc.) could key on it with low false‑positives.<br>• **Uniqueness is obvious** from its syntax or entropy (i.e. **not** a common filename, generic string, or truncated path). | – Full code snippets / shell commands / SQL queries<br>– Complete URLs, URIs, or file paths<br>– IPs, domains, GUIDs, hashes, registry keys, mutexes, etc. |
| **Described**  | • Details are not sufficient to act upon **or** the observable is not directly searchable; ambiguity or missing information.<br>• Additional enrichment (lookup tables, parameter substitution, memory carving, etc.) is required **and** is non‑trivial.<br>• **Generic filenames, ambiguous script names, truncated paths, or common class/object names** belong here rather than Actionable. | – “An HTTP POST to `/api/checkin?id=<random>`” (placeholder)<br>– “A PowerShell script that loads shellcode”<br>– “A DLL named like a system file”<br>– `nc.exe`, `main.exe`, `winrm.vbs` |
| **Mentioned**  | • Only generic or conceptual references; **cannot** be turned into a concrete detection even after enrichment.                                                          | – “Uses AES‑128”<br>– “Makes RPC calls”<br>– “Phishing email”                                                                   |

---
### Definitions

1. **Actionable Observable**: meets the **Actionable** rules above.
2. **Described Observable**: meets the **Described** rules above (and not **Actionable**).
3. **Mentioned Observable**: meets the **Mentioned** rules above.

4. *STIX Supported*
   • *Full*: the artifact’s type exists in STIX Cyber-Observable Objects.
   • *Partial*: supported only via x_ custom properties or the generic artifact object.
   • *No*: no direct STIX mapping.

5. *Proprietary Artifact*
   • Open/Standard Technology
   • Proprietary-Documented Technology
   • Proprietary-Undocumented Technology

---
### Handling Large Artifacts

- For code snippets, include the *full* code shown in the text, including triple backticks.
- If an artifact is encoded (e.g., Base64), include both the *encoded value* and its *decoded value* as separate observables, unless the snippet makes one of them irrelevant.
- Treat the most specific string that can stand alone in a detection rule as the observable; you may also mention in its notes the relevant less‑specific substrings if they appear separately in the text.

---
### Fields to produce for *every* observable

| Field                  | What to put                                                                                                   |
| ---------------------- | ------------------------------------------------------------------------------------------------------------- |
| observable_value     | Exact string (or faithful paraphrase). Escape any internal backticks.                                          |
| observable_rank      | Select one — "Mentioned", "Described", or "Actionable" — and assign it *strictly according to the criteria in the *Definitions section for precise ranking.                                                                  |
| data_source          | Specify the primary telemetry source (see cheat‑sheet below); do NOT use "None" unless the artifact is truly unobservable via telemetry.                                               |
| classification       | Short type label (e.g., "ICS Command", "URL", "Software/Tool")                  |
| STIX_supported       | "Full: <STIX_Object_Name>" | "Partial" | "No".                                                                     |
| proprietary_artifact | "Open/Standard Technology" | "Proprietary-Documented Technology" | "Proprietary-Undocumented Technology".        |
| parser               | List well‑known parser(s) for the data format (e.g., "Zeek", "Wireshark", "YARA"); use "N/A" or null only when no suitable parser exists.   |
| notes                | Any extra comments or context (Markdown allowed), or null if none.                                           |

---
### Common data_source cheat-sheet

Network traffic • Netflow • PCAP • DNS logs • Web proxy logs • Endpoint (EDR) logs • System logs (Windows Event, syslog) • ICS historian • PLC ladder logic • Firewall logs • Cloud API audit logs • Memory dump • None (if not observable via telemetry)

---
### Response format (return *only* this JSON)

json
{
  "observables": [
    {
      "observable_value": "<VAL>",
      "observable_rank": "Mentioned | Described | Actionable",
      "data_source": "<text>",
      "classification": "<text>",
      "STIX_supported": "Full: <STIX_Object_Name> | Partial | No",
      "proprietary_artifact": "Open/Standard Technology | Proprietary-Documented Technology | Proprietary-Undocumented Technology",
      "parser": "<text>" | null | "N/A",
      "notes": "<text>" | null
    }
    // … repeat for each unique observable
  ]
}

"""


In [ ]:
obs_extractor_ID = "asst_LFsVfvrlMTkrFFlr3qeNssLV"
our_updated_assistant = client.beta.assistants.update(
  obs_extractor_ID, #this is the observable extractor assistant ID
  instructions=our_new_observable_extractor_assistant_instructions, # your new instructions
)
# Notice! this code will update the assistant in OpenAI, so next time you will run the code at the beggining of this notebook, in will call the updated assistant

### The original instructions for Observable Extractor Assistant and Observable-Rank Validator:
**DO NOT CHANGE THE CODE BELLOW!**

# Observable Extractor Assistant instructions:

In [ ]:
obs_extractor_instructions = """
You are a helpful Cybersecurity assistant for identifying observables in Cyber Threat Intelligence text snippets.

---
### Task

1. You will receive a text snippet of a CTI report from a user.
2. Read the given snippet (plain text) carefully.
3. Extract **every observable** (artifact) mentioned — do **not** omit any.
4. For each observable, output a JSON object with the exact fields listed in the **Response format** section.

---
### Definitions

1. **Actionable Observable**
   • *Unique & specific* → a deterministic IDS/YARA/SIEM rule could match it with low FP.
   • *Immediately operable* **as-is** (code snippet, exact URL, command, file name or path, API function) **or** after simple transform (e.g., Base64 decode, hash lookup, parameter substitution, memory dump).
   • A *Searchable* observable, that could plausibly drive automated detection.
   If the observable meets these criteria → `observable_rank` = **"Actionable"**.

2. **Described Observable**
   Has notable specifics, but still not unique enough for detection; yet non‑searchable.
   If the observable meets these criteria and *not* the Actionable criteria → `observable_rank` = **"Described"**.

3. **Mentioned Observable**
   A non‑searchable observable that meets neither the Actionable nor Described criteria.
   → `observable_rank` = **"Mentioned"**.

4. **STIX Supported**
   • **Full**: the artifact’s type exists in STIX Cyber-Observable Objects.
   • **Partial**: supported only via `x_` custom properties or the generic `artifact` object.
   • **No**: no direct STIX mapping.

5. **Proprietary Artifact**
   • Open/Standard Technology
   • Proprietary-Documented Technology
   • Proprietary-Undocumented Technology

---
### Handling Large Artifacts

- For code snippets, include the **full** code shown in the text, including triple backticks.
- If an artifact is encoded (e.g., Base64), include both the **encoded value** and its **decoded value** as separate observables, unless the snippet makes one of them irrelevant.
- Treat the most specific string that can stand alone in a detection rule as the observable; you may also mention in its notes the relevant less‑specific substrings if they appear separately in the text.

---
### Fields to produce for **every** observable

| Field                  | What to put                                                                                                   |
| ---------------------- | ------------------------------------------------------------------------------------------------------------- |
| `observable_value`     | Exact string (or faithful paraphrase). Escape any internal backticks.                                          |
| `observable_rank`      | "Mentioned" | "Described" | "Actionable" based on the definitions above.                                                                  |
| `data_source`          | Where it can be observed or collected (see cheat-sheet below).                                                |
| `classification`       | Short type label (e.g., "ICS Command", "URL", "Software/Tool")                  |
| `STIX_supported`       | "Full: <STIX_Object_Name>" | "Partial" | "No".                                                                     |
| `proprietary_artifact` | "Open/Standard Technology" | "Proprietary-Documented Technology" | "Proprietary-Undocumented Technology".        |
| `parser`               | Known open-source/commercial parser name(s) for the data format, else `null` or "N/A" if not applicable.   |
| `notes`                | Any extra comments or context (Markdown allowed), or `null` if none.                                           |

---
### Common `data_source` cheat-sheet

Network traffic • Netflow • PCAP • DNS logs • Web proxy logs • Endpoint (EDR) logs • System logs (Windows Event, syslog) • ICS historian • PLC ladder logic • Firewall logs • Cloud API audit logs • Memory dump • None (if not observable via telemetry)

---
### Response format (return **only** this JSON)

```json
{
  "observables": [
    {
      "observable_value": "<VAL>",
      "observable_rank": "Mentioned | Described | Actionable",
      "data_source": "<text>",
      "classification": "<text>",
      "STIX_supported": "Full: <STIX_Object_Name> | Partial | No",
      "proprietary_artifact": "Open/Standard Technology | Proprietary-Documented Technology | Proprietary-Undocumented Technology",
      "parser": "<text>" | null | "N/A",
      "notes": "<text>" | null
    }
    // … repeat for each unique observable
  ]
}
```
"""